# ANDES Demonstration of `DGPRCTExt` on IEEE 14-Bus System

Prepared by Jinning Wang. Last revised 12 September 2021.

## Background

Voltage signal is set manually to demonstrate `DGPRCTExt`.

In the modified IEEE 14-bus system, 10 `PVD1` are conencted to `Bus4`, and 1 `DGPRCTExt` is added aiming at `PVD1_2`.

## Conclusion

`DGPRCTExt` can be used to implement protection on `DG` models, where the voltage signal can be manipulated manually. This feature allows co-simulation where you can input the external voltage signal into ADNES by `set` function.

In [1]:
import andes
from andes.utils.paths import get_case

andes.config_logger(stream_level=30)

In [2]:
ss = andes.load(get_case('ieee14/ieee14_dgprctext.xlsx'),
                setup=False,
                no_output=True)

Generating code for 1 models on 8 processes.


In [3]:
ss.setup()

True

In [4]:
# use constant power model for PQ
ss.PQ.config.p2p = 1
ss.PQ.config.q2q = 1
ss.PQ.config.p2z = 0
ss.PQ.config.q2z = 0

# turn off under-voltage PQ-to-Z conversion
ss.PQ.pq2z = 0
ss.PFlow.run()

True

## Simulation
Let's run the simulation and manipulate the voltage signal manually.
1) run the TDS to 1s.

In [5]:
ss.TDS.config.tf = 1
ss.TDS.run()

/Users/hcui7/repos/andes/andes/core/model/model.py:790: ComplexWarning: Casting complex values to real discards the imaginary part
  instance.v = np.array(func(*self.s_args[name]),
/Users/hcui7/repos/andes/andes/core/model/model.py:829: ComplexWarning: Casting complex values to real discards the imaginary part
  instance.v[:] = func(*self.s_args[name])


  0%|          | 0/100 [00:00<?, ?%/s]

True

2) store initial Bus4 voltage value.

In [6]:
bus4v0 = ss.Bus.v.v[3]

3) set the external voltage at 0.7 manually.

In [7]:
ss.DGPRCTExt.set(src='v', idx='DGPRCTExt_1', attr='v', value=0.7)

True

4) continue the TDS to 5s.

In [8]:
ss.TDS.config.tf = 5
ss.TDS.run()

  0%|          | 0/100 [00:00<?, ?%/s]

True

5) reset the external voltage back to normal manually.

In [9]:
ss.DGPRCTExt.set(src='v', idx='DGPRCTExt_1', attr='v', value=bus4v0)

True

6) continue the TDS to 10s.

In [10]:
ss.TDS.config.tf = 10
ss.TDS.run()

  0%|          | 0/100 [00:00<?, ?%/s]

True

## Results

### system frequency

In [11]:
ss.TDS.plt.plot(ss.GENROU.omega,
                ycalc=lambda x:60*x,
                title='Generator Speed $\omega$')

/Users/hcui7/repos/andes/andes/plot.py:822: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


(<Figure size 640x480 with 1 Axes>,
 <Axes: title={'center': 'Generator Speed $\\omega$'}, xlabel='Time [s]'>)

### Lock flag

The lock flag is raised at after `TVl1` when the voltage drop below `Vl1`.

In [12]:
ss.TDS.plt.plot(ss.DGPRCTExt.ue,
                title='DGPRCTExt\_1 lock flag (applied on PVD1\_2)')

/Users/hcui7/repos/andes/andes/plot.py:822: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


(<Figure size 640x480 with 1 Axes>,
 <Axes: title={'center': 'DGPRCTExt\\_1 lock flag (applied on PVD1\\_2)'}, xlabel='Time [s]'>)

### PVD1_2 read frequency and frequency signal source

The `PVD1_2` read frequency is locked, but the signal source (in the `BusFreq 4`) remains unchanged

In [13]:
ss.TDS.plt.plot(ss.PVD1.fHz,
                a=(0,1),
                title='PVD1 Read f')

ss.TDS.plt.plot(ss.DGPRCTExt.fHz,
                title='BusFreq 4 Output f')

/Users/hcui7/repos/andes/andes/plot.py:822: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
/Users/hcui7/repos/andes/andes/plot.py:822: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


(<Figure size 640x480 with 1 Axes>,
 <Axes: title={'center': 'BusFreq 4 Output f'}, xlabel='Time [s]'>)

### PVD1_2 power command

`PVD1_2` power commands are locked to 0 **immediately**.

Once the protection was released, they returned to normal **immediately**.

In [14]:
ss.TDS.plt.plot(ss.PVD1.Psum,
                a=(0,1),
                title='PVD1 $P_{tot}$ (active power command)')

ss.TDS.plt.plot(ss.PVD1.Qsum,
                a=(0,1),
                title='PVD1 $Q_{tot}$ (reactive power command)')

/Users/hcui7/repos/andes/andes/plot.py:822: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
/Users/hcui7/repos/andes/andes/plot.py:822: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


(<Figure size 640x480 with 1 Axes>,
 <Axes: title={'center': 'PVD1 $Q_{tot}$ (reactive power command)'}, xlabel='Time [s]'>)

### PVD1_2 current command

Consequently, `PVD1_2` current commands are locked to 0 **immediately**.

Once the protection was released, they returned to normal **immediately**.

In [15]:
ss.TDS.plt.plot(ss.PVD1.Ipul,
                a=(0,1),
                title='PVD1 $I_{p,ul}$ (current command before hard limit)')

ss.TDS.plt.plot(ss.PVD1.Iqul,
                a=(0,1),
                title='PVD1 $I_{q,ul}$ (current command before hard limit)')

/Users/hcui7/repos/andes/andes/plot.py:822: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
/Users/hcui7/repos/andes/andes/plot.py:822: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


(<Figure size 640x480 with 1 Axes>,
 <Axes: title={'center': 'PVD1 $I_{q,ul}$ (current command before hard limit)'}, xlabel='Time [s]'>)

### PVD1_2 output current

As a result, `PVD1_2` output current decreased to 0 **gradually**.

When the protection was released, they returned to normal **gradually**.

Here, the `PVD1` output current `Lag` time constant (`tip` and `tiq`) are modified to 0.5, which is only for observation.

Usually, power electronic device can response in ms level.

In [16]:
ss.TDS.plt.plot(ss.PVD1.Ipout_y,
                a=(0,1),
                title='PVD1 $I_{p,out}$ (actual output current)')

ss.TDS.plt.plot(ss.PVD1.Iqout_y,
                a=(0,1),
                title='PVD1 $I_{q,out}$ (actual output current)')

/Users/hcui7/repos/andes/andes/plot.py:822: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


/Users/hcui7/repos/andes/andes/plot.py:822: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


(<Figure size 640x480 with 1 Axes>,
 <Axes: title={'center': 'PVD1 $I_{q,out}$ (actual output current)'}, xlabel='Time [s]'>)

## Cleanup

In [17]:
!andes misc -C

No output file found in the working directory.
